# Boundary Value Problems

> # Let's begin!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate

#Given prameters
L=4.0    #Length of domain
T=5.0    #Total time
h=0.5    #Space step
k=1.0    #Time step
alpha = 16 # From PDE: u_xx=alpha*u_t
lambda_ = k / (alpha*h**2) #Crank-Nicolson

#Discretization
Nx= int(L/h)  #Number of spatial interval
Nt= int(T/k)  #Number of time steps
x= np.linspace(0, L, Nx+1) #Spatial grid points
t= np.linspace(0, T, Nt+1) #Time grid points

#solution matrix
U= np.zeros((Nx+1, Nt+1)) #U[i,j]=U(ih,jk)

#initial condition
U[:, 0] = (x/2)*(8-x)

#boundary condition
U[0,:]=0
U[-1, :]=8

#Matrix construction for internal nodes
n=Nx -1
A=np.zeros((n,n))
B=np.zeros((n,n))

for i in range(n):
    A[i,i] = 2*(1 + lambda_)
    B[i,i] = 2*(1 - lambda_)
    if i > 0:
        A[i, i-1] = -lambda_
        B[i, i-1] = lambda_
    if i < n - 1:
        A[i, i+1] = -lambda_
        B[i, i+1] = lambda_

#Time-stepping Loop
for j in range(Nt):
    #Extractu_i,j for internal nodes (i = 1 to Nx-1)
    u_prev = U[1:Nx, j]
    #RHS vector
    b = B @ u_prev
    #Add boundary terms to RHS
    b[0] += lambda_*(U[0, j] + U[0, j+1]) #Left Boundary
    b[-1] += lambda_*(U[Nx, j] + U[Nx, j+1]) #Right Boundary

    #Solve A * u_next = b
    u_next = np.linalg.solve(A, b)

    #Update solution
    U[1:Nx, j+1] = u_next

#Table output
header = ["x\\t"]+[f"t={int(ti)}" for ti in t]
table_data = []

for i in range(Nx + 1):
    row = [f"x={x[i]:1f}"]+[round(U[i,j], 4) for j in range(Nt+1)]
    table_data.append(row)

print("\nSolution Table (Explicit FDM):")
print(tabulate(table_data, headers=header, tablefmt= "grid"))